In [ ]:
from datasets import Dataset, load_dataset
task_dataset = load_dataset("json", data_files="./datasets/hellaswag/train.json")["train"]
new_splits = task_dataset.train_test_split(test_size=4000)
new_splits

In [ ]:
train_dataset, dev_dataset = new_splits["train"], new_splits["test"]
train_dataset.to_json("./datasets/hellaswag_36k/train.json")
dev_dataset.to_json("./datasets/hellaswag_4k/train.json")

In [4]:
def get_intervention_locations(share_weights, position, last_position, _first_n, _last_n, layers, pad_mode="last"):

    first_n = min(last_position // 2, _first_n)
    last_n = min(last_position // 2, _last_n)

    pad_amount = (_first_n - first_n) + (_last_n - last_n)
    pad_position = -1 if pad_mode == "first" else last_position
    if share_weights or "+" not in position:
        position_list = [i for i in range(first_n)] + \
            [i for i in range(last_position - last_n, last_position)] + \
            [pad_position for _ in range(pad_amount)]
        intervention_locations = [position_list]*len(layers)
    else:
        left_pad_amount = (_first_n - first_n)
        right_pad_amount = (_last_n - last_n)
        left_intervention_locations = [i for i in range(first_n)] + [pad_position for _ in range(left_pad_amount)]
        right_intervention_locations = [i for i in range(last_position - last_n, last_position)] + \
            [pad_position for _ in range(right_pad_amount)]
        # after padding, there could be still length diff, we need to do another check
        left_len = len(left_intervention_locations)
        right_len = len(right_intervention_locations)
        if left_len > right_len:
            right_intervention_locations += [pad_position for _ in range(left_len-right_len)]
        else:
            left_intervention_locations += [pad_position for _ in range(right_len-left_len)]
        intervention_locations = [left_intervention_locations]*(len(layers)//2) + \
            [right_intervention_locations]*(len(layers)//2)
    
    return intervention_locations

In [9]:
ret = get_intervention_locations(
    share_weights=False, 
    position="f12", 
    last_position=6, 
    _first_n=12, 
    _last_n=0, 
    layers=[5,11,5,11], 
    pad_mode="last"
)
print(ret)

[[0, 1, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6]]
